In [ ]:
import torch
import transformer_model
import pandas as pd
import numpy as np

In [ ]:
#Device set to gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
df = pd.read_csv('data_processed/alphafold_length.csv')
pisces_embeddings = torch.load('data_processed/alphafold_training/embeddings.pt')

In [ ]:
lengths = torch.tensor(df['length'].values)
def create_mask(indices):
    mask = torch.arange(129) < lengths[indices]
    mask = mask.to(device) #Loading the mask to the device
    return mask

### Using the trained models to predict spesific proteins

In [ ]:
#Parameters used for the training
feed_forward_dim1 = 512 #FFNN layer 1
feed_forward_dim2 = 256 #FFNN layer 2
num_epochs = 200
dropout_rate = 0.1

#Embedding dimension coming from Prot-bert
D = 1024

n = 1499
embedding = pisces_embeddings[n,:,:]

model = transformer_model.TransformerModel(embed_dim=D, feed_forward_dim1=feed_forward_dim1, feed_forward_dim2= feed_forward_dim2, dropout_rate = dropout_rate).to(device)
model.load_state_dict(torch.load('predictions_pisces/full_200ep_AdamW/model_postraining_full_200ep_01.pt'))
model.eval()
attention_mask = torch.matmul((create_mask(n)).type(torch.float32).reshape(129,1),torch.ones((1,129)).to(device))
predicted_angles = model.forward(embedding, attention_mask)
angles_new =predicted_angles.T*(180/np.pi)
torch.save(angles_new,"predictions_pisces/full_200ep_AdamW/predicted_angle_alphafold_full_1499.pt")